In [14]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../mypydelta/delta/")
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from handcrafted_features import DocBasedFeatureExtractor, Doc2VecChunkVectorizer, CorpusBasedFeatureExtractor
from utils import get_doc_paths, read_labels

raw_docs_dir = "../data/raw_docs/"
labels_dir = "../data/labels/"
extracted_features_dir = "../data/extracted_features/"

lang = "eng"
doc_paths = get_doc_paths(raw_docs_dir, lang)[:3]

sentences_per_chunk = 200
# d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
# d2vcv.fit_transform(doc_paths)

all_chunk_based_features = []
all_book_based_features = []
all_average_sbert_sentence_embeddings = []
all_doc2vec_chunk_embeddings = []
for doc_path in tqdm(doc_paths):
    fe = DocBasedFeatureExtractor(lang, doc_path, sentences_per_chunk)
    chunk_based_features, book_based_features, average_sbert_sentence_embeddings, doc2vec_chunk_embeddings = fe.get_all_features()
    all_chunk_based_features.extend(chunk_based_features)
    all_book_based_features.append(book_based_features)
    all_average_sbert_sentence_embeddings.append(average_sbert_sentence_embeddings)
    all_doc2vec_chunk_embeddings.append(doc2vec_chunk_embeddings)

  0%|          | 0/3 [00:00<?, ?it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 3/3 [00:14<00:00,  4.84s/it]


In [19]:
cbfe = CorpusBasedFeatureExtractor(lang, doc_paths, all_average_sbert_sentence_embeddings=None, all_doc2vec_chunk_embeddings=None)
# Only use get_absolute_unigram_counts(reset_index=True)
all_corpus_based_features = cbfe.get_all_features().set_index('book_name')
all_corpus_based_features.T.to_csv(extracted_features_dir + lang + '/doc_term_matrix.csv', sep=',')

100%|██████████| 3/3 [00:00<00:00,  7.91it/s]


In [3]:
# book name x words
mfws = all_corpus_based_features
type(mfws)

pandas.core.frame.DataFrame

In [8]:
-mfws.sum()

TypeError: bad operand type for unary -: 'str'

In [24]:
from corpus import Corpus
#The corpus will be saved to a CSV file containing documents in the columns and features in the rows
c = Corpus(file=extracted_features_dir + lang + '/doc_term_matrix.csv')

INFO:corpus:Loading corpus from CSV file ../data/extracted_features/eng/doc_term_matrix.csv ...


<Logger corpus (DEBUG)>
Metadata(complete=True, corpus='../data/extracted_features/eng/doc_term_matrix.csv', frequencies=False, ordered=False, words=None)


In [23]:
c

,the,to,of,and,i,a,in,he,that,you,...,ills,manifest,maturing,inattentively,illuminating,crackled,smokeless,biz,wafting,groundless
Radcliffe_Ann_The-Italian_1797,11691.0,5614.0,6125.0,5158.0,1527.0,2968.0,2002.0,3138.0,2629.0,1459.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Gissing_George_In-the-Year-of-Jubilee_1894,4876.0,3837.0,3659.0,3109.0,3001.0,3462.0,1900.0,1724.0,1754.0,2469.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
Collins_Wilkie_Armadale_1864,19849.0,9626.0,8352.0,8091.0,7193.0,5263.0,6129.0,4438.0,3335.0,3704.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
c.metadata

Metadata(complete=True, corpus='../data/extracted_features/eng/doc_term_matrix.csv', frequencies=False, ordered=True, words=None)

In [ ]:
# absolute values

In [5]:
class MFWTable():
    def __init__(self, mfws):
        # book name x words
        self.mfws = mfws.set_index('book_name', drop=True)
        self.normalized = {}

    def get_normalization(self, normalization):
        
        def zscore(self):
            normalized_mfws = (self.mfws - self.mfws.mean(axis=0))/self.mfws.std(axis=0)
            return normalized_mfws
        
        if normalization in self.normalized:
            normalized_mfws = self.normalized[normalization]
        else:
            if normalization == 'zscore':
                normalized_mfws = zscore()
        
        self.normalized[normalization] = normalized_mfws
        return normalized_mfws
    
n = MFWTable(mfws)
n.get_normalization('zscore')

TypeError: zscore() missing 1 required positional argument: 'self'

In [ ]:
class CompositeDeltaFunction(DeltaFunction):
    """
    A composite delta function consists of a *basis* (which is another delta
    function) and a list of *normalizations*. It first transforms the corpus
    via all the given normalizations in order, and then runs the basis on the
    result.
    """

    def __init__(self, descriptor, name=None, title=None, register=True):
        """
        Creates a new composite delta function.

        Args:
            descriptor (str): Formally defines this delta function. First the
                name of an existing, registered distance function, then, separated
                by ``-``, the names of normalizations to run, in order.
            name (str): Name by which this delta function is registered, in
                addition to the descriptor
            title (str): human-readable title
            register (bool): If true (the default), register this delta
                function on creation
        """
        items = descriptor.split(sep)
        self.basis = registry.deltas[items[0]]
        if hasattr(self.basis, 'metric'):
            self.metric = self.basis.metric
        else:
            self.metric = self.basis.distance_function
        del items[0]
        self.normalizations = [registry.normalizations[n] for n in items]
        super().__init__(self, descriptor, name, title, register)

    def prepare(self, corpus):
        for normalization in self.normalizations:
            corpus = normalization(corpus)
        return corpus

    def __call__(self, corpus):
        return self.create_result(self.basis(self.prepare(corpus)), corpus)

In [ ]:
class _FunctionRegistry():
    def __init__():
        self.deltas = {}
        
    def add_delta(self, delta):
        self.deltas[delta.name] = delta
        
    
registry = _FunctionRegistry()

class Delta():
    def __init__(self, name, normalization, distance):
        self.name = name
        self.normalization = normalization
        self.distance = distance
        # register when instantiated
        registry.add_delta(self)
        
    # call from registry with argument
    def __call__(self, mfws):
        df = pd.DataFrame(index=corpus.index, columns=corpus.index)
        for a, b in combinations(df.index, 2):
            delta = self.distance(corpus.loc[a,:], corpus.loc[b,:], *args, **kwargs)
            df.at[a, b] = delta
            df.at[b, a] = delta
        return df.fillna(0)
    
    def prepare(self, corpus):
        for normalization in self.normalizations:
            corpus = normalization(corpus)
        return corpus

    def __call__(self, corpus):
        return self.create_result(self.basis(self.prepare(corpus)), corpus)

        
Delta('Burrows', 'zscore', 'manhattan')
for delta in registry:
    delta(mfws)